In [1]:
#Import necessary libraries

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import requests

In [3]:
#This is the webscraper that goes through every page on the MoCO Parks website

extensions = []
names = []

for i in np.arange(1, 18, 1):
    #17 pages on MoCo website, so this will go through every page
    url = "https://montgomeryparks.org/parks-trails/parks/page/" + str(i) + "/"
    response = urlopen(Request(url=url, headers={"user-agent": "my-app"}))
    doc = BeautifulSoup(response, "html.parser")

    #Getting headers of page name, so park name
    tags = "h3"
    for i in doc.find_all(tags):
        j = i.string
        names.append(j)
        j1 = j.lower()
        j2 = j1.replace(" ", "-") + "/"
        j3 = j2.replace(".", "").replace("'", "")
        extensions.append(j3)

In [4]:
#Extensions for every park --> attach to https://montgomeryparks.org/parks-and-trails/

extensions_new = []

for i in extensions:
    k = i.split(extensions[-8][6])
    extensions_new.append("".join(k))



extensions = extensions_new
extensions

['aberdeen-local-park/',
 'acorn-urban-park/',
 'adventure-conservation-park/',
 'agricultural-history-farm-park/',
 'amity-drive-neighborhood-park/',
 'aquarius-local-park/',
 'arcola-local-park/',
 'argyle-local-park/',
 'arora-hills-local-park/',
 'aspen-hill-local-park/',
 'avenel-local-park/',
 'ayrlawn-local-park/',
 'batchellors-forest-local-park/',
 'battery-lane-urban-park/',
 'bauer-drive-local-park/',
 'becca-lilly-neighborhood-park/',
 'bedfordshire-neighborhood-park/',
 'bel-pre-neighborhood-park/',
 'bennett-creek-conservation-park/',
 'beret-neighborhood-conservation-area/',
 'berryville-road-neighborhood-conservation-area/',
 'beverly-farms-local-park/',
 'big-pines-local-park/',
 'birch-drive-neighborhood-conservation-park/',
 'black-hill-regional-park/',
 'blair-local-park/',
 'blockhouse-point-conservation-park-&-trails/',
 'blueberry-hill-local-park/',
 'blunt-road-local-park/',
 'bonifant-neighborhood-conservation-park/',
 'booze-creek-stream-valley-park/',
 'bound

In [5]:
#This code cell is meant to save the list *extensions* as a dataframe so it is not lost

extensions_df = pd.DataFrame(data={"Extensions": extensions})
extensions_df.to_csv("extensions.csv")

In [6]:
#Attach url extensions to main url
urls = []

for i in extensions:
    urls.append("https://montgomeryparks.org/parks-and-trails/" + i)

def isascii(s):
    """Check if the characters in string s are in ASCII, U+0-U+7F."""
    return len(s) == len(s.encode())


#Trying to find if there are any urls with special/non-ascii characters that can't be processed
#In this case, it is alright to remove them
for i in np.arange(0, len(urls)):
    if isascii(urls[i]) == False:
        print(i)
        print(urls[i])
        #urls.remove(i)

len(urls)

63
https://montgomeryparks.org/parks-and-trails/caroline-freeland-urban park/
68
https://montgomeryparks.org/parks-and-trails/cedar-island-conservation park/


410

In [7]:
names.remove(names[64])
#names.remove(names[69])

In [8]:
next_remove = names[68]
names.remove(next_remove)

In [9]:
#Removing parks with names that have special characters, now will check to see if *names* and *urls* list match up
len(names) == len(urls)

False

In [10]:
#Get park descriptions for each park

texts = []

for i in urls:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15'}
    r = requests.get(i, headers=headers, allow_redirects=False)
    soup = BeautifulSoup(r.content, 'lxml')
    try:
        tx=soup.find("div", {"class": "p-y"}).get_text()
        texts.append(tx)
    except:
        texts.append("N/A")

In [12]:
texts_df = pd.DataFrame(data={"Name": names, "URL":urls, "Text":texts})
#texts_df.to_csv("texts.csv")

ValueError: arrays must all be same length

In [17]:
texts_df = texts_df[(texts_df["Text"] != "N/A") & (texts_df["Text"] != "\n\n ")].reset_index().drop("index", axis=1)
texts_df

In [29]:
park_info_list = []
amenities=['tennis', 'basketball', 'playground', 'gazebo', 'soccer','volleyball','grills','shelter','trails', 'picnic']

for i in texts_df["Text"]:
    park_info = dict()
    ac = re.search("(\d+\.?\d+)\sacres? | (\d+\.?\d+)\-acre", i)
    if ac == None:
        park_info["size"] = 0
    elif "-" in ac.group(0):
        j = ac.group(0).split("-")
        park_info["size"] = j[0]
    else:
        j = ac.group(0).split(" ")
        park_info["size"] = j[0]

    year = re.search("\s(\d\d\d\d)", i)
    if year == None:
        park_info["year"] = 0
    else:
        park_info["year"] = year.group(0)

    words = i.split()
    for a in amenities:
                # Check if the word is already in dictionary
        if a in words:
                    # Increment count of word by 1
            park_info[a] =  1
        else:
                    # Add the word to dictionary with count 1
            park_info[a] = 0
    park_info_list.append(park_info)

In [30]:
#Creating a large dataframe with park details and amenities
park_info_list_df = pd.DataFrame(park_info_list)
df = pd.concat([texts_df, park_info_list_df], axis=1)
df

In [31]:
#df.to_csv("Park Info Dataframe.csv")

In [85]:
#Time to get addresses for all the parks
#Addresses are enetered in numerous different formats, so we have to use code that is flexible for many cases
park_df = pd.read_csv("/work/Park Info Dataframe.csv")

addresses = []

for i in park_df["URL"]:
    response = urlopen(Request(url=i, headers={"user-agent": "my-app"}))
    doc = BeautifulSoup(response, "html.parser")

    tags1 = "dd"
    g = str(doc.find_all(tags)[1])
    g = g.replace("\n", "")
    g2 = re.findall("\<dd\>\<p\>([\d\s\w]+)\.\<br\>([\d\,\s\w]+)\<\/br\>", g)
    if len(g2) > 0:
        string1 = g2[0][0] + ", " + g2[0][1]
        addresses.append(string1)
    else:
        g3 = re.findall("(\d+\s\w+\s\w+)\.\s(\w+\,\sMD\d{0, 6})", g)
        if len(g3) > 0:
            string2 = g3[0][0] + ", " + g3[0][1]
            addresses.append(string2)
        else:
            g4 = re.findall("\<p\>([\d\s\w]+)\<br\>([\d\w\s\,]+)\<", g)
            if len(g4) > 0:
                string4 = g4[0][0] + ", " + g4[0][1]
                addresses.append(string4)
            else:
                addresses.append(0)


In [78]:
#Getting park identifier codes
#Have to use try/except here because one park has a code that can't be read --> because it's one case, we can
#manually enter it
codes = []

for i in park_df["URL"]:
    response = urlopen(Request(url=i, headers={"user-agent": "my-app"}))
    doc = BeautifulSoup(response, "html.parser")

    try:
        g = doc.find_all("a", {"class": "modal-popup btn"})[0]
        codes.append(re.findall("\?find\=(\w\d\d)\"\s", str(g))[0])
    except:
        codes.append(0)

In [89]:
#Updating park_df dataframe to include addresses and codes
park_df["Address"] = addresses
park_df["Code"] = codes

In [90]:
#Download dataframe
#park_df.to_csv("Park Dataframe.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0e770588-8ebc-4bcb-bac2-a1aa232843cc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>